In [9]:
n = int(input())
if n % 400 == 0:
    print("yes")
elif n % 4 == 0 and n % 100 != 0:
    print("yes")
else:
    print("false")

false


In [ ]:
import numpy as np
class GQA:
    def __init__(self, d_model, num_heads, num_groups, seed=0):
        assert num_heads % num_groups == 0
        self.d_model = d_model
        self.h = num_heads
        self.g = num_groups
        self.d_k = d_model // num_heads
        # Projections
        self.W_Q = np.random.randn(d_model, self.h * self.d_k) / np.sqrt(d_model)
        self.W_K = np.random.randn(d_model, self.g * self.d_k) / np.sqrt(d_model)
        self.W_V = np.random.randn(d_model, self.g * self.d_k) / np.sqrt(d_model)

        # output projection
        self.W_O = np.random.randn(self.h * self.d_k, d_model) / np.sqrt(self.h * self.d_k)

        # mapping head
        self.group_size = self.h // self.g
        self.head2group = (np.arange(self.h) // self.group_size).astype(int)
    
    def softmax(self, x, axis= -1):
        # ổn định số học: trừ đi max để tránh overflow
        x_max = np.max(x, axis=axis, keepdims=True)
        e_x = np.exp(x - x_max)
        return e_x / np.sum(e_x, axis=axis, keepdims=True)
    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        batch, seq_len, _ = x.shape
        # Linear projections
        Q = x @ self.W_Q
        K = x @ self.W_K
        V = x @ self.W_V

        # reshape thành (batch, h, seq_len, d_k) và 
        # (batch, g, seq_len, d_k)
        Q = Q.reshape(batch, seq_len, self.h, self.d_k).transpose(0, 2, 1, 3)
        K = K.reshape(batch, seq_len, self.g, self.d_k).transpose(0, 2, 1, 3)
        V = V.reshape(batch, seq_len, self.g, self.d_k).transpose(0, 2, 1, 3)

        # Attention cho từng head -> ánh xạ sang group
        outputs = []
        for i in range(self.h):
            g = self.head2group[i]

            Qi = Q[:, i, :, :] #(batch, seq_len, d_k)
            